In [1]:
# Load Data
import pandas as pd
import numpy as np
import os

data_path = '../csvs/lab_data_final.csv'
df = pd.read_csv(data_path)

In [2]:
def choose_description(row):
    # if description is not a string, set length to zero
    try:
        length_1 = len(row['Description 1'].split())
    except AttributeError:
        length_1 = 0
    try:
        length_2 = len(row['Description 1'].split())
    except AttributeError:
        length_2 = 0
    
    # return NaN if under 10 words in longest description
    if length_1 < 10 and length_2 < 10:
        return np.nan
    
    if length_1 >= length_2:
        return row['Description 1']
    
    return row['Description 2']

df['Description'] = df.apply(choose_description, axis=1)

In [3]:
def combine_descriptions(row):
    desc_1 = row['Description 1']
    desc_2 = row['Description 2']
    
    # if description is not a string, set it to empty string
    if isinstance(desc_1, float):
        desc_1 = ''
    if isinstance(desc_2, float):
        desc_2 = ''
    
    # return combined descrtiption unless length is zero, then return NaN
    combined_desc = desc_1 + ' ' + desc_2
    return combined_desc if combined_desc != ' ' else np.nan

df['Generated Description'] = df.apply(combine_descriptions, axis=1)

In [4]:
df.head(10)

,id,strain,effect,medical_effect,medical_effect_plain,flavor,terpene,Type,Percent Indica,Percent Sativa,...,CBDV,CBDV-A,delta-9 CBG-A,delta-9 CBG,CBC,Moisture Content,ana360,psilabs,sclabs,Description
0,1,sugar-cane,"Body High, Cerebral, Creative, Energetic, Rela...","Analgesic, Antibacterial, Anti inflammatory","Pain relief, Inhibits bacteria, Reduces inflam...","Candy, Earthy, Fruity, Grape, Herbal, Pungent,...","fruity, sweet, earthy, spicy, other",hybrid,0.40,0.60,...,NaN,NaN,0.660000,NaN,0.020000,NaN,1.0,0.0,0.0,Sugar Cane is a rare slightly sativa dominant ...
1,2,chemdawg,"Cerebral, Creative, Euphoric, Happy, Relaxed, ...","Analgesic, Antibacterial, Antidepressant, Anti...","Pain relief, Inhibits bacteria, Reduces depres...","Chemical, Diesel, Earthy, Pine, Pungent","chemical, earthy, other",hybrid,0.55,0.45,...,NaN,NaN,0.561875,0.315000,0.069000,NaN,19.0,0.0,0.0,With a near-even balance between sativa and in...
2,3,jack-herer,"Body High, Cerebral, Creative, Energetic, Euph...","Analgesic, Antibacterial, Antidepressant, Anti...","Pain relief, Inhibits bacteria, Reduces depres...","Earthy, Herbal, Lemon, Pine, Spicy, Sweet, Woody","citrus, sweet, earthy, spicy",sativa,NaN,NaN,...,0.0000,0.000000,0.678750,0.283824,0.046667,13.000000,114.0,1.0,2.0,Jack Herer is easily one of the best-known str...
3,4,green-dream,"Creative, Energetic, Euphoric, Happy, Hungry, ...","Analgesic, Antiepileptic, Anti inflammatory, A...","Pain relief, Antiepileptic, Reduces inflammati...","Blueberry, Citrus, Earthy, Floral, Fruity, Pin...","fruity, berry, sweet, earthy, other",hybrid,0.50,0.50,...,0.0000,0.000000,0.200000,0.122500,0.016667,NaN,5.0,0.0,1.0,"A sativa dominant hybrid, Green Dream is a cro..."
4,5,lemon-skunk,"Energetic, Euphoric, Focused, Giggly, Happy, R...","Analgesic, Antibacterial, Antidepressant, Anti...","Pain relief, Inhibits bacteria, Reduces depres...","Citrus, Earthy, Lemon, Pungent, Skunk, Sour, S...","citrus, sweet, earthy, other",hybrid,0.40,0.60,...,0.0000,0.000000,0.950000,0.450455,0.054054,10.000000,53.0,2.0,2.0,"With THC levels that reach 22% in some tests, ..."
5,6,girl-scout-cookies,"Cerebral, Creative, Euphoric, Relaxed","Analgesic, Antibacterial, Antiepileptic, Antif...","Pain relief, Inhibits bacteria, Antiepileptic,...","Cherry, Earthy, Lemon, Mint, Nutty, Pungent, S...","fruity, citrus, sweet, earthy, minty, nutty, s...",hybrid,0.60,0.40,...,0.0000,0.001579,0.734000,0.365507,0.078553,11.904286,164.0,15.0,19.0,Girl Scout Cookies is an indica-dominant hybri...
6,7,white-tahoe-cookies,"Aroused, Creative, Euphoric, Happy, Relaxed, S...","Analgesic, Antibacterial, Antidepressant, Anti...","Pain relief, Inhibits bacteria, Reduces depres...","Citrus, Earthy, Fruity, Hash, Herbal, Nutty, S...","fruity, sweet, earthy, nutty, spicy",hybrid,0.70,0.30,...,0.0000,0.000000,1.215000,0.080000,0.000000,NaN,0.0,0.0,2.0,White Tahoe Cookies is an indica dominant hybr...
7,8,sour-diesel,"Creative, Energetic, Euphoric, Happy, Relaxed,...","Analgesic, Antidepressant, Antiemetic, Antiepi...","Pain relief, Reduces depression, Antiemetic, A...","Candy, Citrus, Dank, Diesel, Earthy, Pungent, ...","chemical, sweet, earthy, other",hybrid,0.10,0.90,...,0.0004,0.000400,0.550909,0.266094,0.112268,11.500000,116.0,4.0,26.0,"Sour Diesel, otherwise known as ""Sour D"" or ""S..."
8,9,banana-og,"Body High, Euphoric, Happy, Hungry, Relaxed, S...","Analgesic, Antibacterial, Antidiabetic, Antide...","Pain relief, Inhibits bacteria, Antidiabetic, ...","Earthy, Fruity, Skunk, Sweet, Tropical, Woody","fruity, tropical, sweet, earthy, other",hybrid,0.70,0.30,...,0.0000,0.000000,0.498571,0.277500,0.135000,10.993333,9.0,3.0,2.0,The smell of banana peels and Christmas pine t...
9,10,godfather-og-kush,Relaxed,NaN,NaN,"Earthy, Pepper, Pine, Pungent, Skunk, Spicy","earthy, spicy, other",hybrid,0.50,0.50,...,0.0000,0.000000,0.275000,0.065000,0.040000,NaN,1.0,0.0,1.0,"Godfather OG Kush, also known as “Godfather OG..."


In [5]:
# Load Spacy Model
import spacy

nlp = spacy.load("en_core_web_md")

def tokenize_text(text):
    return nlp(text)

In [6]:
df.flavor = df.flavor.fillna(' ')
df.effect = df.effect.fillna(' ')
df.medical_effect = df.medical_effect.fillna(' ')
df.medical_effect_plain = df.medical_effect_plain.fillna(' ')

df['mass_text'] = (df.strain + df.effect + df.flavor + df.Type + 
                   df.medical_effect + df.medical_effect_plain + 
                   df['Generated Description'])

df.mass_text = df.mass_text.apply(tokenize_text)
df.mass_text[0:2]

0    (sugar, -, caneBody, High, ,, Cerebral, ,, Cre...
1    (chemdawgCerebral, ,, Creative, ,, Euphoric, ,...
Name: mass_text, dtype: object

In [7]:
def get_vector_from_doc(x):
    return x.vector

df['mass_vector'] = df.mass_text.apply(get_vector_from_doc)

vectors = df.mass_vector.apply(pd.Series)

vectors.shape

(1928, 300)

In [ ]:
# Create Tree
from sklearn.neighbors import KDTree

kdtree = KDTree(vectors, leaf_size=30)

In [ ]:
test_string = """Originating from the Hindu Kush mountains near the Afghanistan-Pakistan border, 
Afghan Kush is super relaxing and sleep-inducing. This, too, can help you feel hungry if you’re 
experiencing a lack of appetite, and can relieve pain.""" 

input_vector = get_vector_from_doc(
    tokenize_text(test_string)
)

input_vector = input_vector.reshape(1,-1)

num_matches = 5

dist, ind = kdtree.query(input_vector, k=num_matches, return_distance=True)

In [ ]:
response = df.iloc[ind[0]]
response

In [ ]:
import pickle

with open('kdtree_model_1.1.pkl', 'wb') as f:
    pickle.dump(kdtree, f)

In [ ]:
recommend = np.stack((dist, ind), axis=-1)
recommend